# Training classifier based on manual labels

In [1]:
from langchain_openai import OpenAIEmbeddings
import os
import numpy as np
import pandas as pd

with open("./openai.key", "r") as f:
    os.environ["OPENAI_API_KEY"] = f.read().strip()

models = [
    "text-embedding-ada-002",
    "text-embedding-3-small",
    "text-embedding-3-large",
]

In [2]:
# Load dataset
df = pd.read_feather("./manual_eval_labeled.feather").dropna()
df['sentence'] = df['Prompt'] + ' ' + df['answer']
df['label'] = df['eval']
df = df[['sentence', 'label']]
# Embed dataset for each model
for model in models:
    embeddings = OpenAIEmbeddings(model=model)
    df[model] = embeddings.embed_documents(df['sentence'])
    df[model] = df[model].apply(lambda x: np.array(x, dtype=np.float64))

In [3]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import cohen_kappa_score
import autosklearn.classification
import pickle

res = []

# split data into train and test
for model in models:
    print(model)
    rskf = RepeatedStratifiedKFold(n_splits=2, n_repeats=5, random_state=42)
    for i, (train_index, test_index) in enumerate(rskf.split(list(df[model].values), list(df.label))):
        print(f"Fold {i}")
        # get indices in list
        X_train, X_test = list(df[model].values[train_index]), list(df[model].values[test_index])
        y_train, y_test = list(df.label.values[train_index]), list(df.label.values[test_index])
        # train model
        automl = autosklearn.classification.AutoSklearnClassifier(
            time_left_for_this_task=4*60,
            per_run_time_limit=20,
            n_jobs=-1,
            memory_limit=16*1024,	
        )
        automl.fit(X_train, y_train)
        preds = automl.predict(X_test)
        probas = automl.predict_proba(X_test)
        kappa = cohen_kappa_score(pd.DataFrame(y_test).replace({"R": 0, "H": 1, "W": 2,  "A": 3}), pd.DataFrame(preds).replace({"R": 0, "H": 1, "W": 2,  "A": 3}), weights="quadratic")
        print("Cohens Kappa", kappa)
        res.append({
            "model": model,
            "fold": i,
            "cohen_kappa": kappa,
        })
        # save model
        pickle.dump(automl, open(f"./classification_models/automl_{model}_{i}.pkl", "wb"))

text-embedding-ada-002
Fold 0
[WARNING] [2024-08-27 14:50:17,590:Client-Validation] The provided feature types to autosklearn are of type list.Features have been interpreted as: [(0, Float64Dtype()), (1, Float64Dtype()), (2, Float64Dtype()), (3, Float64Dtype()), (4, Float64Dtype()), (5, Float64Dtype()), (6, Float64Dtype()), (7, Float64Dtype()), (8, Float64Dtype()), (9, Float64Dtype()), (10, Float64Dtype()), (11, Float64Dtype()), (12, Float64Dtype()), (13, Float64Dtype()), (14, Float64Dtype()), (15, Float64Dtype()), (16, Float64Dtype()), (17, Float64Dtype()), (18, Float64Dtype()), (19, Float64Dtype()), (20, Float64Dtype()), (21, Float64Dtype()), (22, Float64Dtype()), (23, Float64Dtype()), (24, Float64Dtype()), (25, Float64Dtype()), (26, Float64Dtype()), (27, Float64Dtype()), (28, Float64Dtype()), (29, Float64Dtype()), (30, Float64Dtype()), (31, Float64Dtype()), (32, Float64Dtype()), (33, Float64Dtype()), (34, Float64Dtype()), (35, Float64Dtype()), (36, Float64Dtype()), (37, Float64Dtype

In [7]:
# collect results
df = pd.DataFrame(res)
# calculate mean and std
aggs = df.groupby("model").agg({"cohen_kappa": ["mean", "std"]})
print(aggs)

                       cohen_kappa          
                              mean       std
model                                       
text-embedding-3-large    0.821845  0.017446
text-embedding-3-small    0.819727  0.021529
text-embedding-ada-002    0.804820  0.014563


In [8]:
# find best model
best = df.sort_values("cohen_kappa", ascending=False).head(1)
best

,model,fold,cohen_kappa
18,text-embedding-3-small,8,0.854005


In [9]:
import pickle
import zlib

# load model
automl = pickle.load(open(f"./classification_models/automl_{best['model'].values[0]}_{best['fold'].values[0]}.pkl", "rb"))
# compress model
compressed = zlib.compress(pickle.dumps(automl))
# save model
with open("./classification_models/best.pkl.zlib", "wb") as f:
    f.write(compressed)